Importando o Drive e dependencias


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import plotly.graph_objects as go

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Importando o dataset

In [0]:
def get_data_train():
    return pd.read_csv("model/train_tratado.csv")

In [0]:
def get_data_test():
  return pd.read_csv("model/test.csv")

In [0]:
enem_train = get_data_train()

In [0]:
enem_test = get_data_test()

Criando dataframe

In [0]:
data_train = pd.DataFrame(enem_train, columns=enem_train.columns)

In [0]:
data_test = pd.DataFrame(enem_test, columns=enem_test.columns)

Tratando os Missing Values

Visualizando dados sem tratamento

In [0]:
features = [
    'NU_NOTA_CN',
    'NU_NOTA_CH',
    'NU_NOTA_LC',
    'NU_NOTA_REDACAO',
    'NU_NOTA_COMP1',
    'NU_NOTA_COMP2',
    'NU_NOTA_COMP3',
    'NU_NOTA_COMP4',
    'NU_NOTA_COMP5']



data_train[features].isnull().sum()

NU_NOTA_CN         3389
NU_NOTA_CH         3389
NU_NOTA_LC         3597
NU_NOTA_REDACAO    3597
NU_NOTA_COMP1      3597
NU_NOTA_COMP2      3597
NU_NOTA_COMP3      3597
NU_NOTA_COMP4      3597
NU_NOTA_COMP5      3597
dtype: int64

Tratar alunos faltantes (considerando zero em todas as outras provas)

In [0]:
data_train = data_train.loc[
      (data_train['NU_NOTA_CN'].notnull())  & (data_train['NU_NOTA_CN'] != 0) & 
      (data_train['NU_NOTA_CH'].notnull())  & (data_train['NU_NOTA_CH'] != 0) &
      (data_train['NU_NOTA_LC'].notnull())  & (data_train['NU_NOTA_LC'] != 0) & 
      (data_train['NU_NOTA_REDACAO'].notnull()) & (data_train['NU_NOTA_REDACAO'] != 0)    
]

In [0]:
data_test = data_test.loc[
      (data_test['NU_NOTA_CN'].notnull())  & (data_test['NU_NOTA_CN'] != 0) & 
      (data_test['NU_NOTA_CH'].notnull())  & (data_test['NU_NOTA_CH'] != 0) &
      (data_test['NU_NOTA_LC'].notnull())  & (data_test['NU_NOTA_LC'] != 0) & 
      (data_test['NU_NOTA_REDACAO'].notnull()) & (data_test['NU_NOTA_REDACAO'] != 0)    
]

Definir dados de testes e treino

In [0]:
# remove a coluna variavel alvo 
X_train = data_train[features]

In [0]:
X_test = data_test[features]

In [0]:
# variável 'y' é nossa variável alvo
y_train = data_train['NU_NOTA_MT']

In [0]:
# imprime a quantidade de linhas dos conjuntos
print ('X_train: numero de linhas e colunas: {}'.format(X_train.shape))
print ('X_test: numero de linhas e colunas: {}'.format(X_test.shape))
print ('y_train: numero de linhas e colunas: {}'.format(y_train.shape))

X_train: numero de linhas e colunas: (9852, 9)
X_test: numero de linhas e colunas: (3286, 9)
y_train: numero de linhas e colunas: (9852,)


In [0]:
# cria um dataframe vazio
df_results = pd.DataFrame()

In [0]:
# cria o objeto rf_regressor
rf_regressor = RandomForestRegressor(
           criterion='mae', 
           max_depth=8,
           max_leaf_nodes=None,
           min_impurity_split=None,
           min_samples_leaf=1,
           min_samples_split=2,
           min_weight_fraction_leaf=0.0,
           n_estimators= 500,
           n_jobs=-1,
           random_state=0,
           verbose=0,
           warm_start=False)

In [0]:
# treina o algoritmo
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [0]:
# adiciona a coluna NU_INSCRICAO
df_results['NU_INSCRICAO'] = data_test['NU_INSCRICAO']

In [0]:
# adiciona os resultados no dataframe df_results
df_results['NU_NOTA_MT'] = rf_regressor.predict(X_test)

In [0]:
df_results.to_csv('answer.csv', index=False)